**Example of double group sequential fit:**

MAKE SURE YOU RENEWED ACCESS TO DATABASE!

In [1]:
%matplotlib tk
from mujpy.musuite import suite
from mujpy.muplot import multiplot
from mujpy.aux.aux import rebin, get_title
import json
from mujpy.mufit import mufit
from mujpy.mufitplot import mufitplot

In [2]:
#  can add 'scan':'T' or 'B' for orderinng csv for increasing T, B, otherwise increasing nrun
input_suite = {'console':'print',
                   'datafile':'/afs/psi.ch/project/bulkmusr/data/gps/d2021/tdc/deltat_tdc_gps_0822.bin',
                   'logpath':'log/',
                   'runlist':'822',
                   'groups calibration':'2-1_3-4.calib',
                   'offset':20
                  }  # 'console':logging, for output in Log Console, 'console':print, for output in notebook
with open('log/input.suite',"w") as f:
    json.dump(input_suite,f)

grp_calib = [{'forward':'2', 'backward':'1', 'alpha':1.135},{'forward':'3', 'backward':'4', 'alpha':1.129}]
with open('log/2-1_3-4.calib',"w") as f:
    json.dump(grp_calib,f)
    
the_suite = suite('log/input.suite',mplot=False) # the_suite implements the class suite according to input.suite

******************* SUITE *********************
Run /afs/psi.ch/project/bulkmusr/data/gps/d2021/tdc/deltat_tdc_gps_0822.bin loaded
Succesfully completed prompt Minuit fit, check nt0, dt0 
****************END OF SUITE*****************


In [3]:
multi = False # preliminary plot skipped
if multi:
    asymm,asyme = the_suite.asymmetry_multigroup()
    if asymm.any() != None:
        t,a = rebin(the_suite.time,asymm,(0,20000),100) 
        nrun = str(the_suite._the_runs_[0][0].get_runNumber_int())
        groups = [grp['forward']+'-'+grp['backward'] for grp in the_suite.groups]
        titles = [nrun+': '+get_title(the_suite._the_runs_[0][0]) for group in groups]
        multi = multiplot(t,a,titles,groups,the_suite.histoLength)
        discard  = multi.display(anim=True)
    else:
        the_suite.console('********************** No data yet: check access to database!')

In [4]:
the_fit = mufit(the_suite,'log/mgml.822.2-1_3-4.1.json')

 _____________________________________________________________________________________
| Run 822: MnBi6Te1 pellet   100.000G 30.000K      on group: 2 - 1   α = 1.135    |
| χᵣ² = 1.041(0.980,1.020), fit on [7.82ns, 9.4µs, 1.95ns/bin], at 29/11/2021 12:55:22|
|-------------------------------------------------------------------------------------|
| mg: A_T_fast = 0.06(1) B_T_fast = 9.9(5) φ_T_fast = -55(2) σ_T_fast = 1.5(4)
| ml: A_T_slow = 0.18(1) B_T_slow = 10.01(1) φ_T_slow = -55(2) λ_T_slow = 0.05(2)
|_____________________________________________________________________________________|
 _____________________________________________________________________________________
| Run 822: MnBi6Te1 pellet   100.000G 30.000K      on group: 3 - 4   α = 1.129    |
| χᵣ² = 1.037(0.980,1.020), fit on [7.82ns, 9.4µs, 1.95ns/bin], at 29/11/2021 12:55:22|
|-------------------------------------------------------------------------------------|
| mg: A_T_fast = 0.058(5) B_T_fast = 10.3(2) φ_T_fast = 3

In [5]:
fit_plot= mufitplot('0,2000,10,20000,40',the_suite,'log/mgml.822.2-1_3-4.1_fit.json')#,guess=True) # '0,1000,4,24000,100' # two range plot

Plotting best fit of run 822
Multigroup animation: toggle pause/resume by clicking on the plot
plot_run mufitplot debug: single not multigroup False, multigroup in comp False


In [6]:
import matplotlib.pyplot as P
from mujpy.aux.aux import rebin
if False:
    t,a,e,f = the_fit.show_calib()
    fig,ax = P.subplots(figsize=(5,4))
    tr, ar, er = rebin(t,a,[0,24000],100,e=e)
    ax.errorbar(tr,ar,yerr=er,fmt='o',ms=0.5,mfc='w',alpha=0.5)
    ax.plot(t,f, 'r-',lw=1,zorder=2)
    P.show()